# 강좌 엑셀 파일에서 불러오기

In [15]:
# from urllib.request import urlopen
# html = urlopen("https://www.frip.co.kr/daily/learning/all?orderBy=popular&page=1")  
# bsObject = BeautifulSoup(html, "html.parser")
# print(bsObject)

In [39]:
import requests
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
url = "https://www.frip.co.kr/products/108923"
webpage = requests.get(url, headers=headers)
soup = BeautifulSoup(webpage.content, "html.parser")
soup.select(
'body')
# print(soup)

[<body><noscript>If you're seeing this message, that means <strong>JavaScript has been disabled on your browser</strong>, please <strong>enable JS</strong> to make this app work.</noscript><div id="app"></div><div id="modal-root"></div><div id="dnd-root"></div><link href="https://fonts.googleapis.com/css?family=Noto+Sans+KR:300,400,500,700" rel="stylesheet"/><script src="/runtime~main.bcf9f3a8eff7fa7c2423.js" type="text/javascript"></script><script src="/vendors~main.3749da4eac8b61e1a652.chunk.js" type="text/javascript"></script><script src="/main.680bb431ba4e89ad202b.chunk.js" type="text/javascript"></script></body>]

In [8]:
with open('lecture.csv') as file:
    csv_data = []
    for line in file.readlines():
        csv_data.append(line.split(','))
print(csv_data[0][0])

강좌명


In [ ]:
import numpy
csv_data = numpy.loadtxt('lecture.csv', delimiter=',')


In [ ]:
import pandas as pd
csv_data = pd.read_csv("./lecture.csv")
print(csv_data)

# 셀리니움을 활용하여 frip에서 크롤링

In [42]:
pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge selenium

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\sehyu\anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    selenium-3.141.0           |py37h8055547_1001         937 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m
  selenium           conda-forge/win-64::selenium-3.141.0-py37h8055547_1001

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.2-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1




p



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [1]:
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime

driver = webdriver.Chrome('./chromedriver')

## 페이지 넘어가면서 크롤링

In [2]:
links = []
for i in range(1, 179):
    driver.get('https://www.frip.co.kr/daily/learning/all?page='+ str(i))
    time.sleep(1)
    lectures = driver.find_elements_by_class_name("sc-kGXeez")
    for lecture in lectures:
        a_name = lecture.find_element_by_tag_name('a').get_attribute('href')
        links.append(a_name)

# 크롤링한 페이지 url csv파일로 저장

## DataFrame 방식으로 저장

In [3]:
import pandas as pd

dataframe = pd.DataFrame(links)
dataframe.to_csv('./frip.csv', header=False,index=False)

## csv writer 방식으로 저장

In [4]:
import csv

In [150]:
f = open('./frip_csv.csv', 'w', newline='')
wr = csv.writer(f)
for link in links:
    wr.writerow([link])
f.close()

# 페이지 url이 저장된 csv 파일 불러오기

In [7]:
f = open('./frip.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [151]:
f = open('./frip_csv.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [9]:
len(new_links)

3554

# 페이지 별로 돌아가면서 데이터 크롤링

## 한개만 테스트

In [12]:
driver.get("https://www.frip.co.kr/products/134289")
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
meeting_id = "https://www.frip.co.kr/products/114211".split('/')[-1]
uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
writer = soup.select_one('span.gYdgbL')
created_at = datetime.now()
updated_at = datetime.now()
is_period = 0
meeting_date = datetime.now()
temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
if temp[1].text[0].isdigit():
    temp_date = ''
    for t in temp[1].text:
        if t == '외':
            break
        if t in '(월화수목금토일요일년)':
            continue
        if t == ' ':
            t = '-'
        temp_date += t
    temp_date = temp_date[:-1]
    meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
period_date = ''
is_class = 1
max_person = 20
now_person = 1
content = '[내용]' + soup.select('div.cSNowP')[0].text
ref_url = new_links[0]
address = soup.select_one('div.sc-kRCAcj').text[:-5]
fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
unit = '원'
is_active = 1
like_temp = ''
for t in temp[0].text:
    if not t.isdigit():
        break
    like_temp += t
like_cnt = int(like_temp)
view_cnt = 0
score = 0
if temp[3].text[6].isdigit():
    score = float(temp[3].text[6:])
main_category = 0
tags = ''

IndexError: string index out of range

In [175]:
meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags
    ]
)

## 미팅 시간 연산

In [85]:
from datetime import datetime

temp_date = ''
for t in temp[1].text:
    if t == '외':
        break
    if t in '(월화수목금토일요일년)':
        continue
    if t == ' ':
        t = '-'
    temp_date += t
temp_date = temp_date[:-1]
datetime_object = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
print(datetime_object)

2020-05-23 13:00:00


## columns 생성

In [25]:
import pandas as pd
from datetime import datetime

meeting_columns = (
    "meeting_id",
    "uid",
    "writer",
    "created_at",
    "updated_at",
    "is_period",
    "meeting_date",
    "period_date",
    "is_class",
    "max_person",
    "now_person",
    "content",
    "ref_url",
    "address",
    "fee",
    "unit",
    "is_active",
    "like_cnt",
    "view_cnt",
    "score",
    "main_category",
    "tags",
    "title",
    "area1",
    "area2"
)

image_columns = (
    "meeting_id",
    "image_url"
)

## 제대로 테스트

In [15]:
meetings = []
images = []

In [32]:
for link in range(246, len(new_links)):
    print(new_links[link])
    driver.get(new_links[link])
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    meeting_id = new_links[link].split('/')[-1]
    uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
    writer = soup.select_one('span.gYdgbL').text
    created_at = datetime.now()
    updated_at = datetime.now()
    is_period = 0
    meeting_date = datetime.now()
    temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
    if temp[-3].text == '':
        continue
    elif temp[-3].text[0].isdigit():
        temp_date = ''
        for t in temp[-3].text:
            if t == '외':
                break
            if t in '(월화수목금토일요일년)':
                continue
            if t == ' ':
                t = '-'
            temp_date += t
        temp_date = temp_date[:-1]
        try:
            meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
        except:
            meeting_date = datetime.now()
    period_date = ''
    is_class = 1
    max_person = 20
    now_person = 1
    content = '[내용]' + soup.select('div.cSNowP')[0].text
    ref_url = new_links[link]
    print(soup.select_one('div.sc-kRCAcj').text[:-5])
    address = soup.select_one('div.sc-kRCAcj').text[:-5]
    fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
    unit = '원'
    is_active = 1
    like_temp = ''
    for t in temp[0].text:
        if not t.isdigit():
            break
        like_temp += t
    like_cnt = int(like_temp)
    view_cnt = 0
    score = 0
    if temp[-1].text[6].isdigit():
        score = float(temp[-1].text[6:])
    main_category = 0
    tags = ''
    title = soup.select_one('h1.jVCVBk').text
    if len(soup.select_one('div.sc-kRCAcj').text[:-5].split(' ')) > 1:
        area1 = soup.select_one('div.sc-kRCAcj').text[:-5].split(' ')[0]
        area2 = soup.select_one('div.sc-kRCAcj').text[:-5].split(' ')[1]
    else:
        area1 = ''
        area2 = ''
    
    # 이미지 저장
    images_link = driver.find_elements_by_class_name('cljRKC')
    image_urls = set()
    for i in images_link:
        image_urls.add(str(i.get_attribute('src')))
    image_url = ' '.join(image_urls)
    
    # 데이터 저장
    meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags,
        title,
        area1,
        area2
        
    ]
    )
        
    images.append(
    [
        meeting_id,
        image_url
    ]
    )


https://www.frip.co.kr/products/111214
서울시 성동구 성수이로12길 6, 2층
https://www.frip.co.kr/products/136225
PDF 전자책입니다
https://www.frip.co.kr/products/115234
서울시 종로구 성균관로19 302호
https://www.frip.co.kr/products/115571
서울시 강남구 논현동 164-14, 3층 GSM테라스(신논현역 3번출구 5분거리)
https://www.frip.co.kr/products/130001
경기도 김포시 장기동 1942-1, 1층 유소도예
https://www.frip.co.kr/products/131853
서울시 서초구 도구로7길 9, 방배동
https://www.frip.co.kr/products/125479
서울시 동작구 동작대로3길 43, 1층 영어공방
https://www.frip.co.kr/products/133886
https://www.frip.co.kr/products/127406
서울 성동구 마조로15가길 2 이엘아트빌 1층
https://www.frip.co.kr/products/111562
서울시 송파구 방이동 156-4, 1층 104호 카페키친
https://www.frip.co.kr/products/133517
서울 성동구 사근동길 6 2층
https://www.frip.co.kr/products/111744
서울시 광진구 건대입구역 근처 스터디룸 (추후 안내 예정)
https://www.frip.co.kr/products/126460
경기도 성남시 분당구 느티로69번길 18, 1층 101호 All Heart
https://www.frip.co.kr/products/135747
서울 마포구 홍익로5길 30 노벨리아서교오피스텔 1010호
https://www.frip.co.kr/products/117914
부산시 남구 대연동 911-5 B1
https://www.frip.co.kr/products/11656

서울시 성동구 성수이로 87, 성문빌딩 1층
https://www.frip.co.kr/products/130319
서울 강서구 마곡중앙로 59-17 류마타워2차 지하1층
https://www.frip.co.kr/products/124209
서울 구로구 오류동 172-67 오남빌딩 지하 102 구루핸디라탄공방
https://www.frip.co.kr/products/134245
경북 칠곡군 왜관읍 남부1길 22 순심여고 강당
https://www.frip.co.kr/products/131187
강남역 인근 스터디룸 (신청하신 분에 한해 추후 안내 드림)
https://www.frip.co.kr/products/111832
서울시 강서구 마곡중앙6로 45 리더스퀘어 B동 6층
https://www.frip.co.kr/products/116624
대전광역시 서구 월평로 13번길 82, 1층 카르페디엠 카페(월평동 419번지)
https://www.frip.co.kr/products/108222
서울시 마포구 월드컵북로6길 19, 성산빌딩 가동 지하1층 (The연습실 2호점)
https://www.frip.co.kr/products/135347
서울시 서대문구 연희동 446-141 아마도
https://www.frip.co.kr/products/133599
서울시 서초구 신반포로23길 41, 잠원쇼핑센터 3층 314호
https://www.frip.co.kr/products/136204
서울시 마포구 토정로25길 31, 1층 리브모아 도예공방
https://www.frip.co.kr/products/136094
서울시 중랑구 동일로140길 65, 1층 본따블르
https://www.frip.co.kr/products/135098
배송상품
https://www.frip.co.kr/products/126704
부산광역시 연제구 월드컵대로 141, 10층
https://www.frip.co.kr/products/135317
서울시 서초구 강남대로 30길 12 은정빌딩 5층

서울 서초구 서초중앙로26길 11 청림빌딩 4층
https://www.frip.co.kr/products/136311
서울시 마포구 합정동 성지길 25-11 지층 (유어 하루드로잉)
https://www.frip.co.kr/products/136221
서울시 강동구 상일로 15길 34-30, 103호
https://www.frip.co.kr/products/108815
서울시 마포구 성산동 58-11, 삼덕빌딩3층 301호
https://www.frip.co.kr/products/109015
서울시 공원 (올림픽공원, 하늘공원, 선유도공원, 서울숲) 등등
https://www.frip.co.kr/products/115482
서울시 서초구 강남대로 30길 12 , 은정빌딩 5층
https://www.frip.co.kr/products/112781
서울시 강남구 봉은사로51길 13 1층 카페데쎄르룸
https://www.frip.co.kr/products/108807
서울시 강서구 화곡동 372-1 3층 301호
https://www.frip.co.kr/products/132015
https://www.frip.co.kr/products/136171
서울 마포구 서교동 457-1 1층 봉봉비
https://www.frip.co.kr/products/134644
서울 강서구 공항대로58길 32 1층 여울정원
https://www.frip.co.kr/products/130743
서울시 강남구 역삼동 657-12, 화향
https://www.frip.co.kr/products/135363
서울시 성동구 성수동1가 656-1095
https://www.frip.co.kr/products/132963
서울시 강동구 암사동 504-29, 지하 1층 우리드리공방
https://www.frip.co.kr/products/127468
서울특별시 서초구 법원로 4길 42 수빌딩 5층 온도르 스튜디오
https://www.frip.co.kr/products/115962
한강윈드서핑장

서울시 중구 충무로7길 17, 2층
https://www.frip.co.kr/products/132861
서울시 중구 세종대로20길 23 원창빌딩 7층 714호 피피아트 스튜디오
https://www.frip.co.kr/products/124292
부산 남구 용소로19번길 15 2층 카페위드
https://www.frip.co.kr/products/135840
온라인
https://www.frip.co.kr/products/133270
온라인
https://www.frip.co.kr/products/132102
서울특별시 서초구 서운로 160, 팔레스빌딩 204호
https://www.frip.co.kr/products/120254
서울시 서대문구 이화여대길 34, 3층 북카페 파오
https://www.frip.co.kr/products/116396
수원 유윤스포츠 (경기도 수원시 팔달구 월드컵로 310)
https://www.frip.co.kr/products/108653
강남역 스터디룸, 신촌역 스터디룸
https://www.frip.co.kr/products/133302
오목교역 2번출구 앞 스터디룸
https://www.frip.co.kr/products/131927
전국 어디서든
https://www.frip.co.kr/products/127541
서울특별시 강남구 봉은사로82길 5 덕봉빌딩
https://www.frip.co.kr/products/134332
서울 서대문구 신촌로 227 3층 아트 스페이스
https://www.frip.co.kr/products/136157
서울시 마포구 양화로8길 16-4, 7층 스페이스T
https://www.frip.co.kr/products/135704
서울 송파구 잠실동 175-14 10층 토즈 스터디 모임공간
https://www.frip.co.kr/products/134576
서울 성동구 성수이로20길 16 JK타워 2층
https://www.frip.co.kr/products/135574
서울시 동작

서울시 중랑구 면목동 428-4 , 201호
https://www.frip.co.kr/products/118019
서울시 용산구 이태원동 92-9번지 3층
https://www.frip.co.kr/products/108341
제주도 서귀포시 고성오조로 108
https://www.frip.co.kr/products/111356
서울시 마포구 연남동 250-5 103호 (연남동)
https://www.frip.co.kr/products/135997
서울 마포구 모래내로3길 11 상암미르웰오피스텔 아담한 홈공방
https://www.frip.co.kr/products/132135
귀뚜라미빌딩 지하 1층 건물 뒤편입구(서울시 강동구 올림픽로 588, 8호선 강동구청역 1번 출구에서 도보 6분)
https://www.frip.co.kr/products/130582
서울 성북구 성북로8길 4 4층
https://www.frip.co.kr/products/135978
서울시 중랑구 동일로140길 65, 1층 본따블르
https://www.frip.co.kr/products/135452
서울 마포구 중동 82-18 1층 아름다운미술마을
https://www.frip.co.kr/products/134828
서울시 마포구 양화로8길 16-4, 7층 스페이스T
https://www.frip.co.kr/products/133888
서울시 은평구 대조동 62-1, 3층 302호 스튜디오 동그란
https://www.frip.co.kr/products/131709
서울 강서구 공항대로58길 32 1층 여울정원
https://www.frip.co.kr/products/133930
서울 관악구 신림로 330 수업 전날 공지
https://www.frip.co.kr/products/125914
서울시 성북구 동소문로15길 36, 1층 귄캔들
https://www.frip.co.kr/products/118035
호스트에게 문의
https://www.frip.co.kr/products/105

ValueError: invalid literal for int() with base 10: ''

In [33]:
meeting_frame = pd.DataFrame(data=meetings, columns=meeting_columns)
meeting_frame

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,unit,is_active,like_cnt,view_cnt,score,main_category,tags,title,area1,area2
0,108923,https://www.frip.co.kr/hosts/617,그 아저씨,2020-05-30 17:44:33.916869,2020-05-30 17:44:33.916869,0,2020-05-30 17:44:33.916869,,1,20,...,원,1,11217,0,4.77,0,,[널위한선물] 도예 공방 원데이 / 도자기 그릇 만들기 (예약가능),서울시,성동구
1,133413,https://www.frip.co.kr/hosts/5545,MCJ,2020-05-30 17:44:37.487207,2020-05-30 17:44:37.487207,0,2020-06-03 19:30:00.000000,,1,20,...,원,1,262,0,5.00,0,,BYOB 각자 와인 1병씩 가져오는 MCJ 홈파티,서울시,강남구
2,101774,https://www.frip.co.kr/hosts/746,아이홉 맥주공방 (I'HOP),2020-05-30 17:44:41.512211,2020-05-30 17:44:41.512211,0,2020-05-31 13:00:00.000000,,1,20,...,원,1,9845,0,4.76,0,,[가정의달] 술술 놀면서 수제 맥주 만들기,서울시,송파구
3,133023,https://www.frip.co.kr/hosts/10654,비푸머스,2020-05-30 17:44:45.015208,2020-05-30 17:44:45.015208,0,2020-05-30 17:44:45.015208,,1,20,...,원,1,886,0,4.81,0,,[서울/홍대] 최상급 프랑스 향료로 나만의 향수 만들기 (예약 가능),서울시,마포구
4,115497,https://www.frip.co.kr/hosts/2839,지아네요리공작소,2020-05-30 17:44:48.477207,2020-05-30 17:44:48.477207,0,2020-05-30 17:44:48.477207,,1,20,...,원,1,5675,0,4.89,0,,[망원동] 스콘 2종 원데이클래스 (예약 가능),서울시,마포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,113810,https://www.frip.co.kr/hosts/6528,PPT마스터,2020-05-30 19:34:25.954343,2020-05-30 19:34:25.954343,0,2020-06-06 11:00:00.000000,,1,20,...,원,1,1216,0,4.94,0,,상위 1%의 광고인이 알려주는 PPT신공!,강남역,인근
812,120929,https://www.frip.co.kr/hosts/7621,봉봉주스,2020-05-30 19:34:29.694343,2020-05-30 19:34:29.694343,0,2020-05-30 19:34:29.694343,,1,20,...,원,1,257,0,4.91,0,,"커피의 모든 것, 추출부터 라떼아트까지 (예약 가능)",서울,영등포구
813,109373,https://www.frip.co.kr/hosts/3456,샤당구클럽,2020-05-30 19:34:33.432348,2020-05-30 19:34:33.432348,0,2020-05-30 19:34:33.432348,,1,20,...,원,1,547,0,4.81,0,,1:1 포켓볼 집중 강습! (예약 가능),서울시,관악구
814,115452,https://www.frip.co.kr/hosts/242,에코다이버스,2020-05-30 19:34:38.200345,2020-05-30 19:34:38.200345,0,2020-05-30 19:34:38.200345,,1,20,...,원,1,140,0,0.50,0,,[제주] 신비로운 제주 바다! 1박2일 다이빙 자격증 완파하기!,제주도,서귀포시


In [35]:
meeting_frame.to_csv('./meeting_data.csv', header=True,index=False, encoding='utf-8-sig')

In [36]:
meeting_frame.tail(100)

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,unit,is_active,like_cnt,view_cnt,score,main_category,tags,title,area1,area2
716,124335,https://www.frip.co.kr/hosts/8394,예소피아노,2020-05-30 19:27:55.160937,2020-05-30 19:27:55.160937,0,2020-05-30 19:27:55.160937,,1,20,...,원,1,321,0,4.96,0,,[NEW최신가요출시] 똥꼬발랄 예소쌤과 대환장 피아노원데이클래스 1일1곡 완성시키기...,서울시,영등포구
717,108410,https://www.frip.co.kr/hosts/5324,김아이캘리그라피,2020-05-30 19:27:59.582936,2020-05-30 19:27:59.582936,0,2020-05-30 19:27:59.582936,,1,20,...,원,1,997,0,4.90,0,,[원데이] 손글씨로 나만의 캘리그라피 액자 만들기 (예약 가능),홍대입구역,근처
718,112131,https://www.frip.co.kr/hosts/500,유어 하루드로잉,2020-05-30 19:28:02.391937,2020-05-30 19:28:02.391937,0,2020-06-02 19:30:00.000000,,1,20,...,원,1,5400,0,4.89,0,,여행지 펜 드로잉,서울시,마포구
719,115615,https://www.frip.co.kr/hosts/1609,앙센베이킹,2020-05-30 19:28:07.051248,2020-05-30 19:28:07.051248,0,2020-05-30 19:28:07.051248,,1,20,...,원,1,219,0,4.86,0,,레몬 케이크 만들기 (예약 가능),서울시,중랑구
720,118019,https://www.frip.co.kr/hosts/519,레더레시피🌿,2020-05-30 19:28:11.400245,2020-05-30 19:28:11.400245,0,2020-05-30 19:28:11.400245,,1,20,...,원,1,197,0,4.83,0,,직접 만드는 가죽 여권케이스 (예약 가능),서울시,용산구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,113810,https://www.frip.co.kr/hosts/6528,PPT마스터,2020-05-30 19:34:25.954343,2020-05-30 19:34:25.954343,0,2020-06-06 11:00:00.000000,,1,20,...,원,1,1216,0,4.94,0,,상위 1%의 광고인이 알려주는 PPT신공!,강남역,인근
812,120929,https://www.frip.co.kr/hosts/7621,봉봉주스,2020-05-30 19:34:29.694343,2020-05-30 19:34:29.694343,0,2020-05-30 19:34:29.694343,,1,20,...,원,1,257,0,4.91,0,,"커피의 모든 것, 추출부터 라떼아트까지 (예약 가능)",서울,영등포구
813,109373,https://www.frip.co.kr/hosts/3456,샤당구클럽,2020-05-30 19:34:33.432348,2020-05-30 19:34:33.432348,0,2020-05-30 19:34:33.432348,,1,20,...,원,1,547,0,4.81,0,,1:1 포켓볼 집중 강습! (예약 가능),서울시,관악구
814,115452,https://www.frip.co.kr/hosts/242,에코다이버스,2020-05-30 19:34:38.200345,2020-05-30 19:34:38.200345,0,2020-05-30 19:34:38.200345,,1,20,...,원,1,140,0,0.50,0,,[제주] 신비로운 제주 바다! 1박2일 다이빙 자격증 완파하기!,제주도,서귀포시


# 이미지 데이터 저장

In [37]:
image_frame = pd.DataFrame(data=images, columns=image_columns)

In [38]:
image_frame = pd.read_csv('./meeting_images.csv')

In [24]:
image_frame.to_csv('./meeting_images.csv', header=True, encoding='utf-8-sig')

In [10]:
meeting_frame = pd.read_csv('./meeting_data.csv')

In [25]:
meeting_frame.dtypes

meeting_id         int64
uid               object
writer            object
created_at        object
updated_at        object
is_period          int64
meeting_date      object
period_date      float64
is_class           int64
max_person         int64
now_person         int64
content           object
ref_url           object
address           object
fee                int64
unit              object
is_active          int64
like_cnt           int64
view_cnt           int64
score            float64
main_category      int64
tags             float64
dtype: object

# writer 이름 이상했던거 수정(이제 필요 없음)

In [6]:
writers = []
for i, m in meeting_frame.iterrows():
    tmp = m.writer[31:]
    tmp = tmp[:-7]
    writers.append(tmp)
writers

['그 아저씨',
 'MCJ',
 "아이홉 맥주공방 (I'HOP)",
 '비푸머스',
 '지아네요리공작소',
 '포임스포츠클럽',
 '정원속화실담다',
 '트렌드바이미플라워',
 '29 x 냥초',
 '___알렉스',
 '빨간머리 앤',
 "아이홉 맥주공방 (I'HOP)",
 '이도경',
 '방쿤',
 '정원속화실담다',
 '제이몬티 베이킹랩',
 '비비우드스튜디오',
 'SO뮤직',
 '이지훈',
 '한국술집 안씨막걸리',
 'HappyBaker',
 '강남클라이밍',
 '남대문실탄사격장',
 '파파쿡',
 '이타(ITA)',
 '오하나_메이',
 '제이몬티 베이킹랩',
 '공유카페',
 '추월',
 '공유카페',
 '트렌드바이미플라워',
 '예빈당',
 '보니보니베이킹스튜디오',
 '본따블르',
 'Art Space',
 '권영세클라이밍센터',
 '블러썸앤어나더',
 'mok',
 'OvenPal',
 '디세',
 'OvenPal',
 '솔메',
 '이든',
 '쩡쌤',
 '현우',
 '브라보서프',
 '공유카페',
 '여우학교',
 '롤앤쿠키',
 '도란도자기공방',
 '스무:디 (20diy)',
 '샤뜰리에화실',
 '조수정폴댄스',
 'Ray Kim',
 '아브릴 베이킹 스튜디오',
 '레이나스튜디오',
 'POTTRING',
 '청년건물주 제이든',
 '프롬디센트',
 '다미',
 '벨로르플라워',
 '와플목공방',
 '헤펠레목공방',
 '피피아트',
 '당근자판기',
 '타이거 포터리',
 'WM트릭킹짐',
 'dojagihaeum',
 '담따프레시',
 '카이카스튜디오',
 '재테크 神',
 'simcook',
 '리브모아',
 '스튜디오 글로위',
 '보니보니베이킹스튜디오',
 'NPC 지니',
 'flake',
 '보니보니베이킹스튜디오',
 'Anna',
 'Stan',
 '호비뮤직',
 '드로잉클럽',
 '방쿤',
 'Tenuto',
 '전상면',
 '유일향',
 '두부',
 '아브릴 베이킹 스튜디오',
 '카이카스튜디오',
 '마크컬렉션'